# Model Evaluation and Selection

Content:

- Regression

- Classification

- Classification with SMOTE


Central Concepts:

- ...

ToDo:

- Add automl lib

- Add hyperparameter-tuning lib


---
Model Evaluation and Selection
- bias variance trade off / over fitting - under fitting
- the curse of complexity
- K-fold Cross Validation
- Ranking 
- ROC / AUC
- Cumulative Percentile Hits
---

In [ ]:
# matplotlib color settings

dark_plot_theme = True

if dark_plot_theme:
    plt.style.use('dark_background')

In [ ]:
# pandas display settings

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [ ]:
# load general dependencies
from collections import Counter
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
import requests


# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# data preparation
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif

# regression metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import d2_tweedie_score

# clustering metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# regression models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# classification models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
# notebook display settings

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)


In [ ]:
# prepare the working directory

cwd = Path()

ipath = cwd / 'data'

ipath.mkdir(exist_ok=True)

ifile = ipath / 'features.csv'


# Load the Dataset

In [ ]:
# import raw dataset

cindex = ['id']

data = pd.read_csv(ifile, index_col=cindex).sort_index(axis=1)

---
---
---

# Regression

- use the log transform
- calculate withdata z_score
- feature selection
- prepare for dataliers
- log transformation - reverse

## Feature Selection

In [ ]:
target = 'price'
x = np.log(data.drop(target, axis=1).select_dtypes(include=np.number) + 1)
y = np.log(data[target] + 1)

selector = SelectKBest(f_regression, k=3)
features_selected = selector.fit_transform(x, y)

In [ ]:
selection_dict = {}
selection_dict['score'] = [round(s, 3) for s in selector.scores_]
selection_dict['name'] = selector.feature_names_in_
selection_dict['p-value'] = selector.pvalues_

selection = pd.DataFrame(selection_dict)
display(selection.sort_values('score', ascending=False).head(10))

In [ ]:
# prepare data

# set the number of used features
n = 3
vs = selection.sort_values('score', ascending=False).head(n)['name'].values

# log transformation?
x = np.log(data[vs] + 1)
y = np.log(data[target] + 1)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

## Model Application

In [ ]:
# regression algorithms and metrics
rs = 42

regs = {}
regs['LinearRegression'] = LinearRegression()
regs['DecisionTreeRegressor'] = DecisionTreeRegressor(random_state=rs)
regs['RandomForestRegressor'] = RandomForestRegressor(random_state=rs)
regs['AdaBoostRegressor'] = AdaBoostRegressor(random_state=rs)
regs['GradientBoostingRegressor'] = GradientBoostingRegressor(random_state=rs)

rmetrics = {}
rmetrics['r2_score'] = r2_score
rmetrics['mean_squared_error'] = mean_squared_error
rmetrics['explained_variance_score'] = explained_variance_score
rmetrics['d2_tweedie_score'] = d2_tweedie_score

In [ ]:
# regression calculation

regression_data = {}
for cname, reg in regs.items():
    reg.fit(xtrain, ytrain)
    ypred = reg.predict(xtest)

    classifier_data = {}
    for mname, metric in rmetrics.items():
        classifier_data[mname] = metric(ytest, ypred)

    regression_data[cname] = classifier_data

regression_results = pd.DataFrame(regression_data)
display(regression_results.T.sort_values('r2_score', ascending=False))


---

# Classification

- Logistic Regression
- Decision Tree
- Random Forest
- Ada Boost Tree
- Gradient Boosting Tree

- define the classification problem
- airbnb more expensive then mean
- short excurse
- show the ROC-Curve AUC-Score
- validate on test set
- use smote

In [ ]:
def get_classification_target(target):
    return (target > np.percentile(target, 75)).astype(int)

## Feature Selection

In [ ]:
target = 'price'
x = np.log(data.drop(target, axis=1).select_dtypes(include=np.number) + 1)
y = np.log(get_classification_target(data[target]) + 1)

selector = SelectKBest(f_classif, k=3)
features_selected = selector.fit_transform(x, y)

In [ ]:
selection_dict = {}
selection_dict['score'] = [round(s, 3) for s in selector.scores_]
selection_dict['name'] = selector.feature_names_in_
selection_dict['p-value'] = selector.pvalues_

selection = pd.DataFrame(selection_dict)
display(selection.sort_values('score', ascending=False).head(10))

In [ ]:
# prepare data

# set the number of used features
n = 5
vs = selection.sort_values('score', ascending=False).head(n)['name'].values

x = data[vs]
y = get_classification_target(data[target])

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)


## Model Application

In [ ]:
# classification algorithms and metrics
rs = 42

classifiers = {}
classifiers['LogisticRegression'] = LogisticRegression(random_state=rs)
classifiers['DecisionTreeClassifier'] = DecisionTreeClassifier(random_state=rs)
classifiers['RandomForestClassifier'] = RandomForestClassifier(random_state=rs)
classifiers['AdaBoostClassifier'] = AdaBoostClassifier(random_state=rs)
classifiers['GradientBoostingClassifier'] = GradientBoostingClassifier(random_state=rs)

cmetrics = {}
cmetrics['accuracy_score'] = accuracy_score
cmetrics['f1_score'] = f1_score
cmetrics['precision_score'] = precision_score
cmetrics['recall_score'] = recall_score

In [ ]:
# classification

classification_data = {}
for cname, reg in classifiers.items():
    reg.fit(xtrain, ytrain)
    ypred = reg.predict(xtest)
    
    classifier_data = {}
    for mname, metric in cmetrics.items():
        classifier_data[mname] = metric(ytest, ypred)

    classification_data[cname] = classifier_data

test_results = pd.DataFrame(classification_data)
display(test_results.T.sort_values('f1_score', ascending=False))


---

## Classification with SMOTE
Synthetic Minority Oversampling Technique

In [ ]:
# prepare data with SMOTE

_, xtest, _, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

print('Original dataset shape %s' % Counter(y))

sm = SMOTE(random_state=42)

x_smote, y_smote = sm.fit_resample(x, y)

xtrain, _, ytrain, _ = train_test_split(x_smote, y_smote, test_size=0.2, random_state=42)


print('Resampled dataset shape %s' % Counter(y_smote))

In [ ]:
# classification with smote

classification_data = {}
for cname, reg in classifiers.items():
    reg.fit(xtrain, ytrain)
    ypred = reg.predict(xtest)
    
    classifier_data = {}
    for mname, metric in cmetrics.items():
        classifier_data[mname] = metric(ytest, ypred)

    classification_data[cname] = classifier_data

test_results = pd.DataFrame(classification_data)
display(test_results.T.sort_values('f1_score', ascending=False))

---
---
---